# Find fuel prices
from various WWW

In [160]:
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import requests
import re
import numpy as np
from datetime import datetime as dt
from datetime import date
import sqlalchemy as sa
from itertools import chain

%run '../serv_energy.py'

cstrEIABrent_spot=r'https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=PET&s=RBRTE&f={0}'
cstrEIAWTI_spot=r'https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=PET&s=RWTC&f={0}'

#print(get_date_list(1987)) # check import serv_energy

#conn=sqlite3.connect('fuels.sqlite3')
eng_co=sa.create_engine('sqlite+pysqlite:///fuels.sqlite3') #, echo=True)

at this address - http://www.worldbank.org/en/research/commodity-markets - is a XLSX-file with monthly prices for resources from the World Bank. OIL, COAL, GAS ets.

Next func read and parse this file, return ALL columns

In [133]:
strWB=r'http://www.worldbank.org/en/research/commodity-markets'

def read_wb_prices():
    sessWB=requests.session()
    wb_html=sessWB.get(strWB)
    wb_html.encoding=wb_html.apparent_encoding
    wb_soup=BeautifulSoup(wb_html.text, 'html.parser')

    str_price_pattern=re.compile('Monthly prices', re.MULTILINE)
    url_prices=wb_soup('a', string=str_price_pattern)[0]['href']
    pdfWBPrices=pd.read_excel(r'/home/egor/Загрузки/CMOHistoricalDataMonthly.xlsx', sheet_name='Monthly Prices', 
                          header=6, na_values='..', index_col=0)
    pdfWBPrices.index.name='Date'
    pdfWBPrices.index=pd.to_datetime(pdfWBPrices.index, format='%YM%m')
    pdfWBPrices.applymap(lambda x: float(x))
    return pdfWBPrices

## OIL
Brent, WTI, spot and futures
### Sources: 
1. https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=PET&s=RBRTE&f=M

### BRENT & WTI, spot, dollars per barrel
source: USA EIA (Energy Information Administration)

In [77]:
dctFrequency={'Daily':'D', 'Weekly':'W', 'Monthly':'M', 'Year':'a'}

def EIA_spot(strURL, strName):
    r=requests.get(strURL)
    soup=BeautifulSoup(r.text, 'html.parser')
    tbls=soup.findAll('table', class_='FloatTitle')

    sPrices=pd.Series()
    for tr in tbls[0].findAll('tr'):
        lstTD=[i.text.strip() for i in tr.findAll('td')]
        if len(lstTD)>1:
            dctD=dict(zip(get_date_list(int(lstTD[0])), list(map(lambda x: x if x!='' else np.nan, lstTD[1:]))))
            sPrices=sPrices.append(pd.Series(dctD))
    sPrices.name=strName
    return sPrices.dropna()

In [78]:
#getBRENT().to_sql('BRENT_spot', con=conn, if_exists='replace', index_label='DATE')
brent=EIA_spot(cstrEIABrent_spot.format(dctFrequency['Monthly']), 'BRENT')
wti=EIA_spot(cstrEIAWTI_spot.format(dctFrequency['Monthly']), 'WTI')
    
print_last_point('WTI', wti.index[-1], wti.iloc[-1])
print_last_point('BRENT', brent.index[-1], brent.iloc[-1])

WTI: last date = Aug-2018, last val = 68.06
BRENT: last date = Aug-2018, last val = 72.53


### write readed, merjed and sorted data to OIL_SPOT table

In [79]:
pdfOIL=FuelsDataFrame(brent).merge(FuelsDataFrame(wti), how='outer', left_index=True, right_index=True)
pdfOIL.index.name='Date'
pdfOIL=pdfOIL.sort_index_date(inplace=False)

#pdfOIL.to_sql('OIL_SPOT', eng_co, if_exists='upsert')
#print(pdfOIL.tail(5))

          BRENT    WTI
Date                  
Apr-2018  72.11  66.25
May-2018  76.98  69.98
Jun-2018  74.41  67.87
Jul-2018  74.25  70.98
Aug-2018  72.53  68.06


## OIL from WorldBank file
Brent, Dubai, WTI - without URALS

In [209]:
pdfOils=read_wb_prices().filter(regex='CRUDE').dropna(axis=0, how='all')
oils=FuelsDataFrame(pdfOils)
oils.index_to_string()
oils.to_sql('OILS', eng_co, if_exists='upsert')
print(oils.tail(5))

/home/egor/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


          CRUDE_PETRO  CRUDE_BRENT  CRUDE_DUBAI  CRUDE_WTI
Date                                                      
May-2018    73.430000        76.65        73.66      69.98
Jun-2018    71.976667        75.19        73.22      67.52
Jul-2018    72.666667        74.44        72.72      70.84
Aug-2018    71.083333        73.13        72.13      67.99
Sep-2018    75.363333        78.86        77.02      70.21


## Urals
Source - http://economy.gov.ru/minec/activity/sections/foreignEconomicActivity/monitoring/index?WCM_PI=1&WCM_PageSize.556a41804309ce5898fbbb1aee474279=40 - base URL from MinEconom RF.
It will be a little tricky:
1. Read URL above
2. Find in WWW key words 'нефть Urals'.
3. Go to URL of this item
4. Read URL price (considering date of publishing)
5. Read URALS price from this URL (and date)
6. Upsert URALS table in Database (or CREATE for first time). If CREATE - read ALL items from MinEco and... Everything is difficult there. Many formats, so it's easy make table dy hand (timeline not so long)

In [119]:
#lItemsOnPage=1000 # for CREATING Urals table
lItemsOnPage=40 # for UPSERT Urals table

strEconomyGovRU=r'http://economy.gov.ru/minec/activity/sections/foreignEconomicActivity/monitoring/index?WCM_PI=1&WCM_PageSize.556a41804309ce5898fbbb1aee474279={0}'

#class="docs_bookmarks"
sess1=requests.session()
docs=sess1.get(strEconomyGovRU.format(lItemsOnPage))
docs.encoding=docs.apparent_encoding
spEcon=BeautifulSoup(docs.text, 'html.parser')

tblDocs=spEcon.findAll('table', {'class': re.compile('list_docs\d+')})

print('All Items in site ', len(tblDocs))
print('Page title = ', spEcon.find('title').text)

All Items in site  40
Page title =  Министерство экономического развития Российской Федерации - Деятельность - Конъюнктура мировых товарных рынков


In [120]:
dfMinEcoItems=pd.DataFrame(columns=['Date', 'Title', 'URL'])

for i, tbl in enumerate(tblDocs):
    dfMinEcoItems.loc[i]=[tbl.find('span', class_='list_docs_date').text.strip(), 
                          tbl.find('span', class_='list_docs_title').text.strip(),
                         tbl.find('a')['href']]

print(dfMinEcoItems)
print(dfMinEcoItems[dfMinEcoItems['Title'].str.contains(r'([нН]ефть)|(Юралс)')])

             Date                                              Title  \
0   03.10.2018 г.  О средней цене на нефть сорта «Юралс» за сентя...   
1   10.09.2018 г.  Сообщение о средних за истекший налоговый пери...   
2   04.09.2018 г.  О средней цене на нефть сорта «Юралс» за авгус...   
3   15.08.2018 г.  Сообщение о средних за истекший налоговый пери...   
4   13.08.2018 г.  О средней цене на нефть сорта «Юралс» за июль ...   
5   13.07.2018 г.  Сообщение о средних за истекший налоговый пери...   
6   04.07.2018 г.  О средней цене на нефть сорта «Юралс» за июнь ...   
7   14.06.2018 г.  Сообщение о средних за истекший налоговый пери...   
8   07.06.2018 г.  О средней цене на нефть сорта «Юралс» за май 2...   
9   15.05.2018 г.  Сообщение о средних за истекший налоговый пери...   
10  07.05.2018 г.  О средней цене на нефть сорта «Юралс» за апрел...   
11  12.04.2018 г.  Сообщение о средних за истекший налоговый пери...   
12  12.04.2018 г.  О средней цене на нефть сорта «Юралс» за март

/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


In [84]:
# CHEATING

pdf=pd.read_csv(r'urals.csv', sep=';', encoding='cp1251')

dctTr={'Январь':'Jan', 'Февраль':'Feb', 'Март':'Mar', 'Апрель':'Apr', 'Май':'May', 'Июнь':'Jun', 'Июль':'Jul', 
 'Август':'Aug', 'Сентябрь':'Sep', 'Октябрь':'Okt', 'Ноябрь':'Nov', 'Декабрь':'Dec'}

pdf['Date']=pdf.apply(lambda x: '{0}-{1}'.format(dctMonthTranslate[x['Month']], x['Year']), axis=1)
pdfUrals=pdf[['Date', 'val']].rename(columns={'val':'Price'})

urals_hist=FuelsDataFrame(pdfUrals)
urals_hist.set_index('Date', inplace=True)
print(urals_hist.tail(10))
urals_hist=urals_hist.sort_index_date(inplace=False)


urals_hist.to_sql('URALS', eng_co, if_exists='replace') # make URALS sqlite table

print(urals_hist.shape)

          Price
Date           
Dec-2017   63.8
Jan-2018   68.6
Feb-2018   63.2
Mar-2018   63.6
Apr-2018   69.1
May-2018   75.1
Jun-2018   73.1
Jul-2018   72.8
Aug-2018   71.5
Sep-2018   76.3
(117, 1)


In [121]:
#filter items with oil prices
pdfOilItems=dfMinEcoItems[dfMinEcoItems['Title'].str.contains(r'([нН]ефть)|(Юралс)')] 

strReplNew=r'http://economy.gov.ru/minec/activity/sections/'
strReplOld=r'/wps/wcm/connect/economylib4/mer/activity/sections/'

def get_date_from_title(strTitle):
    strSearch=r'(?i).*(?P<month>' + '|'.join(dctTr.keys()) + ')\s(?P<year>\d{4}).*'
    #print(strSearch)
    mo=re.match(strSearch, strTitle)
    return '{0}-{1}'.format(dctMonthTranslate[mo[1].capitalize()], mo[2])
   
def get_price(soup):
    str_price_pattern=re.compile('^\d+,?\d*\sдолл\. США.*', re.MULTILINE)
    strFind=soup('p', string=str_price_pattern)[0].text.replace(',', '.')
    #print(strFind)
    return re.match(r'\d+\.?\d*', strFind).group(0)

pdfUralsPrice=pd.DataFrame(columns=['Date', 'Price'])

#get urals prices and date from items

sess2=requests.session()

for i, Item in pdfOilItems.iterrows():
    htmlI=sess2.get(Item['URL'].replace(strReplOld, strReplNew))
    htmlI.encoding=htmlI.apparent_encoding
    u_i=BeautifulSoup(htmlI.text, 'html.parser')
    str_title=u_i.find('title')
    try:
        pdfUralsPrice.loc[i]=[get_date_from_title(str_title.text), get_price(u_i)]
    except:
        print(str_title.text)
        print(Item['URL'].replace(strReplOld, strReplNew))
        print('\n')
    

print(pdfUralsPrice)

/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


        Date  Price
0   Sep-2018  78.13
2   Aug-2018  71.52
4   Jul-2018  72.84
6   Jun-2018  73.06
8   May-2018  75.14
10  Apr-2018  69.10
12  Mar-2018  63.58
14  Feb-2018  63.14
16  Jan-2018  68.60
18  Dec-2017  63.77


### For UPSERT URALS Database table

In [122]:
pdfNewPrice=FuelsDataFrame(pdfUralsPrice)
pdfNewPrice.set_index('Date', inplace=True)

pdfNewPrice=pdfNewPrice.sort_index_date()
print(pdfNewPrice)

          Price
Date           
Dec-2017  63.77
Jan-2018  68.60
Feb-2018  63.14
Mar-2018  63.58
Apr-2018  69.10
May-2018  75.14
Jun-2018  73.06
Jul-2018  72.84
Aug-2018  71.52
Sep-2018  78.13


In [123]:
pdfNewPrice.to_sql('URALS', eng_co, if_exists='upsert')

## Coal

### https://ycharts.com/indicators/australia_coal_price 
Australia energy, USD per Metric Ton, F.O.B. piers, Newcastle/Port Kembla; 6,300 kcal per Kilogram (11,340 btu/lb), less than 0.8%, sulfur 13% ash beginning January 2002; previously 6,667 kcal per Kilogram (12,000 btu/lb), less than 1.0% sulfur, 14% ash.

In [172]:
strAusCoal=r'https://ycharts.com/indicators/australia_coal_price'

def australian_energy(strURL=strAusCoal):
    sessY1=requests.session()
    aus_coal=sessY1.get(strAusCoal)
    aus_coal.encoding=aus_coal.apparent_encoding
    spAC=BeautifulSoup(aus_coal.text, 'html.parser')

    tblACPrices=spAC.findAll('table', class_='histDataTable')
    
    dctACPrices=dict()

    for tbl in tblACPrices:
        tr=tbl.findAll('tr')
        for r in tr:
            strDt=r.find('td', class_='col1')
        
            if strDt is not None:
                tmpDt=strDt.text.strip().split(' ')
                strDt=tmpDt[0][:3].lower()+'-'+tmpDt[2]
                dtPr=r.find('td', class_='col2').text.strip()
                dctACPrices.setdefault(strDt, dtPr)
    srACPrices=pd.Series(dctACPrices)
    srACPrices.name='AustralianEnergy'
    srACPrices.index.name='Date'
    return FuelsDataFrame(srACPrices)

#pdfAC=australian_energy().sort_index_date()
#print(pdfAC.tail(2))

         AustralianEnergy
Date                     
aug-2018           117.34
sep-2018           114.16


COALS from WorldBank file (Australian, Colombian, South Afirca

In [208]:
pdfCoals=read_wb_prices().filter(regex='COAL').dropna(axis=0, how='all')
coals=FuelsDataFrame(pdfCoals)
coals.index_to_string()
coals.to_sql('COALS', eng_co, if_exists='upsert')
print(coals.tail(5))

/home/egor/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


          COAL_AUS  COAL_COL  COAL_SAFRICA
Date                                      
May-2018    106.20       NaN        101.89
Jun-2018    114.75       NaN        103.88
Jul-2018    119.57       NaN        106.02
Aug-2018    117.34       NaN         98.48
Sep-2018    114.16       NaN        102.14


### NYMEX coal futures 
https://www.eia.gov/coal/markets/includes/nymex_price_data_historical.xlsx

In [266]:
strNYMEX_COAL=r'https://www.eia.gov/coal/markets/includes/nymex_price_data_historical.xlsx'
def read_nymex_coal(strURL=strNYMEX_COAL):
    # short ton to tonne - 0.907185
    k_con=0.907185
    pdfNC=pd.read_excel(strURL)
    pdfNC['Date']=pd.to_datetime(pdfNC['DATA_DATE'], format='%Y-%m-%d')
    pdfg=pdfNC.set_index('Date').groupby(pd.Grouper(freq='M'))[['EASTERN_RAIL', 'WESTERN_RAIL', 'INDONESIAN_MCCLOSKEY', 
                                                                'NYMEX_APPALACHIAN']].mean()
    return pdfg/k_con
  
nymex_coal=FuelsDataFrame(read_nymex_coal())
nymex_coal.index_to_string()
nymex_coal.to_sql('NYMEX_COALS', eng_co, if_exists='upsert')
print(nymex_coal.tail(10))

          EASTERN_RAIL  WESTERN_RAIL  INDONESIAN_MCCLOSKEY  NYMEX_APPALACHIAN
Date                                                                         
Jan-2018     71.229762     13.700676                   NaN                NaN
Feb-2018     71.027119     14.067229                   NaN                NaN
Mar-2018     69.739017     13.980978                   NaN                NaN
Apr-2018     65.594328     13.986752                   NaN                NaN
May-2018     63.425472     13.741309                   NaN                NaN
Jun-2018     67.126540     13.790960                   NaN                NaN
Jul-2018     70.359986     13.638736                   NaN                NaN
Aug-2018     72.992158     13.730961                   NaN                NaN
Sep-2018     78.081909     13.672137                   NaN                NaN
Oct-2018     85.291313     13.778887                   NaN                NaN


# Currency prices


In [260]:
dct_Currency_translate={'USD':['14', 'USD (Доллар)'],
'EUR':['7', '(EUR (Евро)'],
'CHF':['4', 'CHF (Швейцарский Франк)'],
'GBP':['8', 'GBP (Английский Фунт Стерлингов)'],
'AMD':['18', 'AMD (Армянский Драм)'],
'AUD':['1', 'AUD (Австралийский Доллар)'],
'AZN':['19', 'AZN (Азербайджанский Манат)'],
'BGN':['2', 'BGN (Болгарский Лев)'],
'HUF':['9', 'HUF (Венгерский Форинт)'],
'BRL':['31', 'BRL (Бразильский реал)'],
'BYN':['20', 'BYN (Белорусский Рубль)'],
'CAD':['3', 'CAD (Канадский Доллар)'],
'CNY':['21', 'CNY (Китайский юань Ренминби)'],
'CZK':['5', 'CZK (Чешская Крона)'],
'DKK':['6', 'DKK (Датская Крона)'],
'INR':['22', 'INR (Индийская pупия)'],
'JPY':['10', 'JPY (Японская Иена)'],
'KGS':['23', 'KGS (Киргизский Сом)'],
'KRW':['24', 'KRW (Южнокорейский вон)'],
'KZT':['25', 'KZT (Казахский Тенге)'],
'MDL':['17', 'MDL (Молдавский лей)'],
'NOK':['26', 'NOK (Норвежская Крона)'],
'PLN':['11', 'PLN (Польская Злота)'],
'RON':['15', 'RON (Румынский Лей)'],
'SEK':['12', 'SEK (Шведская Крона)'],
'SGD':['32', 'SGD (Сингапурский доллар)'],
'TJS':['27', 'TJS (Таджикский Сомони)'],
'TMT':['28', 'TMT (Туркменский Манат)'],
'TRY':['13', 'TRY (Турецкая Лира)'],
'UAH':['16', 'UAH (Украинская Гривна)'],
'UZS':['29', 'UZS (Узбекский Сум)'],
'ZAR':['33', 'ZAR (Южноафриканский рэнд)']}

#print(dct_Currency_translate)

pdfCurrencyDict=pd.DataFrame.from_dict(dct_Currency_translate, orient='index', columns=['Value', 'Info'])
pdfCurrencyDict.index.name='Code'
#pdfCurrencyDict.to_sql('curr_digest', eng_co, if_exists='replace')

print(pdfCurrencyDict.head(3))

     Value                     Info
Code                               
USD     14             USD (Доллар)
EUR      7              (EUR (Евро)
CHF      4  CHF (Швейцарский Франк)


In [309]:
strWWW_Price=r'https://www.kursvaliut.ru/%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B8%D0%B9-%D0%BE%D0%B1%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9-%D0%BA%D1%83%D1%80%D1%81-%D0%B7%D0%B0-%D0%BC%D0%B5%D1%81%D1%8F%D1%86'
lStartYear=2003

def get_currency(name='USD', strURL=strWWW_Price, year=2018):
    def get_year_prices(sess, lYear):
        html=sess.post(strURL, data={'currency':str(pdfCurrencyDict.loc[name]['Value']), 'year':str(lYear), 'display':''})
        html.encoding=html.apparent_encoding
        soup=BeautifulSoup(html.text, 'html.parser')
        tbl_curr=soup.findAll('table', class_='table table-condensed table-borderless') 
        dctCurr=dict()
        
        for r in tbl_curr[0].findAll('tr'):
            tds=r.findAll('td')
            if len(tds)==3:
                dctCurr.setdefault('{0}-{1}'.format( dctMonthTranslate[tds[1].text], lYear), 
                                   tds[2].text.replace(',', '.').replace('RUB', '').strip())
        return pd.Series(dctCurr).replace('-', np.nan).dropna()
    
    sessCUR=requests.session()
    pdSer=None
    
    if type(year) is list:
        pdSer=pd.concat([get_year_prices(sessCUR, str(y)) for y in year])
    else:
        pdSer=get_year_prices(sessCUR, str(year))
        
    pdSer.name=name
    pdSer.index.name='Date'
    return pdSer

#print(get_currency(year=list(range(lStartYear, dt.now().year+1))))
print(get_currency(year=dt.now().year))


Date
Jan-2018    56.4981
Feb-2018    56.8067
Mar-2018    57.0636
Apr-2018    60.7699
May-2018    62.2309
Jun-2018    62.7685
Jul-2018    62.8619
Aug-2018    66.0764
Sep-2018    67.6661
Oct-2018    65.6081
Name: USD, dtype: object


In [288]:
pdf=pdfCurrencyDict.loc[['USD', 'EUR', 'CHF', 'GBP', 'ZAR', 'AUD', 'UAH', 'TRY', 'CNY']]
lstSerCurr = [get_currency(name=n, only_last=True) for n in pdf.index.tolist()]

pdfAllCurrency=pd.concat(lstSerCurr, axis=1)
print(pdfAllCurrency.head(10))

              USD      EUR      CHF      GBP      ZAR      AUD      UAH  \
Apr-2003  31.2118  33.8671  22.6552  49.1350      NaN  19.0203  58.0955   
Apr-2004  28.6856  34.4463  22.1445  51.8878      NaN  21.4323  53.4004   
Apr-2005  27.8104  35.9935  23.2550  52.6926      NaN  21.4981  53.6130   
Apr-2006  27.5645  33.7674  21.4333  48.6204      NaN  20.2415  54.1084   
Apr-2007  25.8384  34.8919  21.3074  51.3488      NaN  21.3404  51.0500   
Apr-2008  23.5129  37.0635  23.2486  46.5852      NaN  21.8549  46.8334   
Apr-2009  33.5833  44.2798  29.2092  49.2375      NaN  23.8827  41.7683   
Apr-2010  29.1932  39.2297  27.3569  44.6934  39.7876  27.0111  36.7309   
Apr-2011  28.0840  40.5525  31.2502  45.9490  41.7470  29.6854  35.2378   
Apr-2012  29.4909  38.8145  32.2850  47.1885  37.6668  30.5124  36.7276   

              TRY      CNY  
Apr-2003      NaN      NaN  
Apr-2004      NaN      NaN  
Apr-2005  20.4012      NaN  
Apr-2006  20.5983      NaN  
Apr-2007  18.9809  33.4447  


/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [291]:
#print(pdfAllCurrency)

currency=FuelsDataFrame(pdfAllCurrency)
currency=currency.sort_index_date()
currency.index.name='Date'

currency.to_sql('CURRENCY', eng_co, if_exists='replace')

print(currency)

              USD      EUR      CHF      GBP      ZAR      AUD      UAH  \
Date                                                                      
Jan-2003  31.8162  33.8068  23.1122  51.4064      NaN  18.5728  59.4571   
Feb-2003  31.6990  34.1878  23.2872  51.2838      NaN  18.8274  59.1510   
Mar-2003  31.4533  33.9520  23.1135  49.7896      NaN  18.9436  58.7159   
Apr-2003  31.2118  33.8671  22.6552  49.1350      NaN  19.0203  58.0955   
May-2003  30.9071  35.7382  23.5722  50.1619      NaN  20.0244  56.9717   
Jun-2003  30.4686  35.5942  23.1201  50.5924      NaN  20.2379  56.2599   
Jul-2003  30.3603  34.5600  22.3280  49.3934      NaN  20.1325  56.6282   
Aug-2003  30.3490  33.8762  21.9774  48.4264      NaN  19.7733  56.3997   
Sep-2003  30.5986  34.2997  22.1499  49.1956      NaN  20.2222  56.8598   
Oct-2003  30.1647  35.2963  22.7996  50.5292      NaN  20.8875  55.9533   
Nov-2003  29.8080  34.9241  22.4059  50.4194      NaN  21.3480  55.3104   
Dec-2003  29.4337  36.133

## Uranium Price

https://www.cameco.com/invest/markets/uranium-price

In [239]:
strUraniumURL=r'https://www.cameco.com/invest/markets/uranium-price'


def get_uranium(strURL=strUraniumURL):
    sess=requests.session()
    ura=sess.get(strURL)
    ura.encoding=ura.apparent_encoding
    sp_ura=BeautifulSoup(ura.text, 'html.parser')
    
    tbls=sp_ura.findAll('table', class_='table-data')
    #tbls[0] - spot prices
    ths=[th.text for th in tbls[0].findAll('th')]
    ths[0]='Month'
    tds=[[td.text for td in tr.findAll('td')] for tr in tbls[0].findAll('tr') ]
    pdfUra=pd.DataFrame(list(filter(None, tds)), columns=ths).set_index('Month')
    pdfUra=pdfUra.stack().reset_index()
    pdfUra['Date']=pdfUra.apply(lambda x: '{0}-{1}'.format(x['Month'].capitalize(), x['level_1']), axis=1)
    pdfUra.rename(columns={0:'U3O8'}, inplace=True)
    pdfU=FuelsDataFrame(pdfUra[['Date', 'U3O8']].set_index('Date')).replace({'-':np.nan}).dropna()
    #print(pdfU)
    return pdfU.sort_index_date(inplace=False)

    #print(list(filter(None, tds)))
    #print(len(tbls))

uranium=get_uranium()
uranium.to_sql('URANIUM', eng_co, if_exists='upsert')
print(uranium.tail(5))

           U3O8
Date           
May-2018  22.73
Jun-2018  22.65
Jul-2018  25.78
Aug-2018  26.30
Sep-2018  27.50


## Natural gas

### Henry hub - https://www.eia.gov/dnav/ng/hist/rngwhhdm.htm

In [241]:
strHenryHubURL=r'https://www.eia.gov/dnav/ng/hist/rngwhhdm.htm'

def get_henry_nub(strHHURL=strHenryHubURL):
    def convert2float(strDigit):
        try:
            return float(strDigit)
        except:
            return np.nan
        
    sess=requests.session()
    hh=sess.get(strHHURL)
    hh.encoding=hh.apparent_encoding
    sp_hh=BeautifulSoup(hh.text, 'html.parser')
    
    tbl=sp_hh.find('th', text='Year').parent.parent
    lstCols=[h.text for h in tbl.findAll('th')]
    lstCels=list(filter(None, [[convert2float(td.text) for td in r.findAll('td')] for r in tbl.findAll('tr')]))
    pdf=pd.DataFrame(lstCels, columns=lstCols).dropna(how='all').set_index('Year').unstack().reset_index()
    pdf.rename(columns={0:'NG_HH', 'level_0':'month'}, inplace=True)
    pdf['Date']=pdf.apply(lambda x: '{}-{}'.format(x['month'].lower(), int(x['Year'])), axis=1)
    pdf.set_index('Date', inplace=True)
    pdf.index=pdf.index.str.capitalize()
    return FuelsDataFrame(pdf[['NG_HH']]).sort_index_date(inplace=False).dropna()
    
henry_hub=get_henry_nub()
print(henry_hub.head(5))

          NG_HH
Date           
Jan-1997   3.45
Feb-1997   2.15
Mar-1997   1.89
Apr-1997   2.03
May-1997   2.25


### NG - YChart and WorldBank -  'for Europe'
https://ycharts.com/indicators/europe_natural_gas_price

In [242]:
strCalcGas=r'https://ycharts.com/indicators/europe_natural_gas_price'

def get_ychart_gas(strURL=strCalcGas):
    def correct_date(strDT):
        lst=strDT.split(' ')
        lst[0]=lst[0][:3]
        return ' '.join(lst)
    
    sess=requests.session()
    eg=sess.get(strURL)
    eg.encoding=eg.apparent_encoding
    sp_eg=BeautifulSoup(eg.text, 'html.parser')
    
    tbls=sp_eg.findAll('table', class_='histDataTable')
    
    trs=list(chain.from_iterable([t.findAll('tr') for t in tbls])) 
    #lstVals=list(filter(None, [[td.text.strip() for td in tr.findAll('td')] for tr in tbls[0].findAll('tr')]))
    lstVals=list(filter(None, [[td.text.strip() for td in tr.findAll('td')] for tr in trs]))
    pdf=pd.DataFrame(lstVals, columns=['Date', 'NG_EUROPE'])
    pdf['Date']=pdf['Date'].apply(lambda x: dt.strptime(correct_date(x), '%b %d, %Y').strftime('%b-%Y').lower())
    return FuelsDataFrame(pdf.set_index('Date'))

def get_WB_prices():
    pdfWBGas=read_wb_prices().filter(regex='GAS').dropna(axis=0, how='all')
    pdfWBG=FuelsDataFrame(pdfWBGas)
    pdfWBG.index_to_string()
    pdfWBG.index=pdfWBG.index.str.lower()
    return pdfWBG

ychart_gas=get_ychart_gas()
wb_gas=get_WB_prices()['NGAS_EUR']
wb_gas.update(ychart_gas['NG_EUROPE'])
wb_gas.dropna(inplace=True)
wb_gas.name='NG_EUROPE'
europe_gas=FuelsDataFrame(wb_gas)
europe_gas.index=europe_gas.index.str.capitalize()
print(europe_gas.tail(10))
    

         NG_EUROPE
Date              
Aug-2017      5.28
Sep-2017      5.51
Oct-2017      6.08
Nov-2017      6.02
Dec-2017      6.56
Jan-2018      7.56
Feb-2018      6.87
Mar-2018      7.03
Apr-2018      7.81
May-2018      7.19


In [245]:
nat_gas=pd.merge(henry_hub, europe_gas, how='outer', left_index=True, right_index=True).sort_index_date(inplace=False)
nat_gas.to_sql('NATURAL_GAS', eng_co, if_exists='upsert')
print(nat_gas.tail(10))

          NG_HH NG_EUROPE
Date                     
Dec-2017   2.82      6.56
Jan-2018   3.87      7.56
Feb-2018   2.67      6.87
Mar-2018   2.69      7.03
Apr-2018   2.80      7.81
May-2018   2.80      7.19
Jun-2018   2.97       NaN
Jul-2018   2.83       NaN
Aug-2018   2.96       NaN
Sep-2018   3.00       NaN


## LNG
https://bluegoldresearch.com/global-lng-prices - https://bluegoldresearch.com/datadownload?chnid=410

In [232]:
strLNG_Prices=r'https://bluegoldresearch.com/datadownload?chnid=410'

pdLNG=FuelsDataFrame(pd.read_csv(strLNG_Prices, index_col='Date', parse_dates=True)).rename(columns={'United Kingdom':'UK'})
pdLNG.index_to_string()
pdLNG.to_sql('LNG', eng_co, if_exists='upsert')
print(pdLNG.tail(10))

          Japan  China   UK  Spain  Brazil
Date                                      
Dec-2017   10.0    9.9  8.0    9.3     9.4
Jan-2018   10.9   10.9  7.3    8.0    10.3
Feb-2018    9.5    9.4  7.0    7.2     9.4
Mar-2018    7.9    7.8  6.9    7.2     7.7
Apr-2018    7.2    7.1  6.7    6.9     7.1
May-2018    8.4    8.4  7.4    7.9     7.9
Jun-2018   10.3   10.2  7.4    8.1     9.8
Jul-2018   10.2   10.2  7.5    8.2     9.8
Aug-2018   10.4   10.4  7.9    8.6    10.0
Sep-2018   11.6   11.6  9.5    9.9    11.4


## Petroleum and Diesel

### Russia, Ukraine, Belarussia, Moldova
https://www.benzin-price.ru/price.php

In [248]:
lStartYear=2008

#strPetrolBaseURL=r'https://www.benzin-price.ru/stat_month.php?month={month}&year={year}&region_id={region}'
strPetrolBaseURL=r'http://www.benzin-price.ru/stat_month.php?month={month}&year={year}&region_id={region}'
strPetroMainURL=r'https://www.benzin-price.ru'

strPetrol2=r'http://prices.gasoil.media/fuel/dynamics/?fuel=Ai_92&period=month&subject=213'

req_headers={'accept': 'text/plain, text/html',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'cache-control': 'max-age=0',
'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/69.0.3497.81 Chrome/69.0.3497.81 Safari/537.36'}



def get_regions(strURL=strPetroMainURL):
    sessPet=requests.session()
    print(sessPet.cookies)
    sessPet.headers.update(req_headers)
    
    htmlPet=sessPet.get(strURL)
    print(strURL)
    print(sessPet.headers)
    print(sessPet.cookies)
    htmlPet.encoding=htmlPet.apparent_encoding
   
    soupPet=BeautifulSoup(htmlPet.text, 'html.parser')

    #h3b=soupPet.find('h3', string='Выберите регион')
    #print(h3b.text)
    return sessPet, soupPet

sessionPetrol, soupPet=get_regions()
print(soupPet.title)

<RequestsCookieJar[]>
https://www.benzin-price.ru
{'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/69.0.3497.81 Chrome/69.0.3497.81 Safari/537.36', 'accept-encoding': 'gzip, deflate, br', 'accept': 'text/plain, text/html', 'Connection': 'keep-alive', 'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7', 'cache-control': 'max-age=0'}
<RequestsCookieJar[<Cookie ck=f24ec981d09f103016154725b0bc9174 for .benzin-price.ru/>, <Cookie sn=2c6f8c967d45b5a81d753fdd215fba1f for .benzin-price.ru/>, <Cookie PHPSESSID=2c6f8c967d45b5a81d753fdd215fba1f for www.benzin-price.ru/>]>
None


In [12]:
a_regs=soupPet.findAll('a', {'href':re.compile(r'/price.php\?region_id')})

dctRegionCodes={re.search('(\d+)$', a['href']).group(0):re.match('\D+', a.text).group(0) for a in a_regs}

pdfRegCodes=pd.Series(dctRegionCodes)
pdfRegCodes.name='RegionName'
pdfRegCodes.index.name='Code'
#pdfRegCodes.to_sql('regcodes_digest', eng_co, if_exists='replace')
print(len(pdfRegCodes))

0


In [268]:
from stem import Signal
from stem.control import Controller
from time import sleep
import socket
import socks

cur_month=9
cur_region=77
cur_year=2018

#socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, "127.0.0.1", 9050, True)
#socket.socket = socks.socksocket
#controller=Controller.from_port(port=9051, address='127.0.0.1')

lstUA=[r'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
r'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0', 
      r'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41',
      r'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1',
      r'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)',
       r'Googlebot/2.1 (+http://www.google.com/bot.html)'
      ]

strPetrolBaseURL1=r'https://www.benzin-price.ru/stat_month.php?month={month}&year={year}&region_id={region}'
strPetroMainURL1=r'https://www.benzin-price.ru'

def cnange_tor_proxy():
    #socket.setdefaulttimeout(5)
    
    with Controller.from_port(address='127.0.0.1', port=9051) as controller:
        controller.authenticate(password='cmasf')
        controller.signal(Signal.NEWNYM)
        sleep(10)
        session=requests.session()
        hdrs=req_headers
        hdrs['user-agent']=lstUA[4]
        session.headers=hdrs
        session.proxies={'http': 'socks5://127.0.0.1:9050',
                        'https': 'socks5://127.0.0.1:9050' }
        r=session.get('http://icanhazip.com/')
        print('new IP is', r.text.strip())
        return session


def get_month_prices(current_month=10, current_year=2018, current_region=77, session=sessionPetrol):
    def make_soup():
        #print(strPetrolBaseURL.format(month=current_month, region=current_region, year=current_year))
        #print(session.headers)
        #session.cookies.clear()
        #hdrs=req_headers
        #hdrs['user-agent']=lstUA[3]
        #r=session.get(strPetroMainURL1)
        #sleep(10)
        #if len(r.text)!=0:
        #    print('response ', r.status_code)
        #    print('headers ', r.headers)
        #    print('cookies ', r.cookies)
        
        #print(r, session.cookies)
        #sleep(5)
        htmlPet=session.get(strPetrolBaseURL1.format(month=current_month, region=current_region, year=current_year))
        #print('response ', htmlPet.status_code)
        #print('headers ', htmlPet.headers)
        #print('cookies ', htmlPet.cookies)
        htmlPet.encoding=htmlPet.apparent_encoding
        return BeautifulSoup(htmlPet.text, 'html.parser')
        
    num_trying=0

    lstHeaders=[]
    lstPrices=[]
    
    print('start read for {0} month; {1} year; {2} region'.format(current_month, current_year, current_region), end=' ... ')
    
    #print(session.headers)
    
    while num_trying<6:
        soupPetPrice=make_soup()
        try:
            td_work=soupPetPrice.find('td', class_='content')
            tblWorks=td_work.findAll('table')
            lstHeaders=[th.text for th in tblWorks[0].findAll('th')]
            lstPrices=list(filter(None, [[td.text.strip() for td in tr.findAll('td')] for tr in tblWorks[0].findAll('tr')]))
            break
        except:
            if num_trying==5:
                print('fatal error reading!!!')
                return None
            num_trying+=1
            sec_wait=30*num_trying
            print('error reading {0} - sleep and wait {1} sec... '.format(num_trying, sec_wait), end=' ')
            sleep(sec_wait)
            session=cnange_tor_proxy()
   
    pdfMonth=pd.DataFrame(lstPrices, columns=lstHeaders).set_index('период')
    pdfMonth=pdfMonth.applymap(lambda x: float(x) if x !='' else np.nan)
    pdfMonth=pdfMonth.mean(axis=0)
    pdfMonth.name=date(current_year, current_month, 1).strftime('%b.%Y')
    pdfMonth['region']=current_region
    print(' parsed ', pdfMonth.shape)
    sleep(15)
    return pdfMonth

#print(get_month_prices())

tmp1=get_month_prices(current_month=9, current_year=2011, current_region=77, session=cnange_tor_proxy())
tmp2=get_month_prices(current_month=10, current_year=2016, current_region=77, session=cnange_tor_proxy())

#cnange_tor_proxy()
print(tmp1)

new IP is 65.19.167.130
start read for 9 month; 2011 year; 77 region ... error reading 1 - sleep and wait 30 sec...  new IP is 185.24.218.174
 parsed  (13,)
new IP is 185.220.101.6
start read for 10 month; 2016 year; 77 region ... error reading 1 - sleep and wait 30 sec...  new IP is 91.219.236.171
error reading 2 - sleep and wait 60 sec...  new IP is 197.231.221.211
error reading 3 - sleep and wait 90 sec...  new IP is 109.248.9.8
error reading 4 - sleep and wait 120 sec...  new IP is 92.63.103.241
error reading 5 - sleep and wait 150 sec...  new IP is 185.220.101.33
fatal error reading!!!
80        24.525000
92        26.980000
92+             NaN
95        28.925000
95+       30.055000
98        31.386667
98+       32.937500
100             NaN
100+            NaN
ДТ        26.367500
ДТ+       27.282500
Газ       16.575000
region    77.000000
Name: Sep.2011, dtype: float64


In [237]:
#print(pd.concat([tmp1, tmp2], axis=1).T)
#tmpErr=get_month_prices(current_month=1, current_year=lStartYear, current_region=77)
#print(tmpErr)

pdfRegCodes=pd.read_sql('regcodes_digest', con=eng_co, index_col='Code')
print(pdfRegCodes.head(10))
print(pdfRegCodes.index.tolist())

#for r in pdfRegCodes.loc[pdfRegCodes.index.str.len()< 3].index.tolist():
#    print(r)

                             RegionName
Code                                   
77                            г. Москва
50    Московская область и новая Москва
78                   г. Санкт-Петербург
47                Ленинградская область
22                       Алтайский край
28                     Амурская область
29                Архангельская область
30                 Астраханская область
31                 Белгородская область
32                     Брянская область
['77', '50', '78', '47', '22', '28', '29', '30', '31', '32', '33', '34', '35', '36', '79', '75', '37', '38', '7', '39', '40', '41', '42', '43', '44', '23', '24', '45', '46', '48', '49', '51', '83', '52', '53', '54', '55', '56', '57', '58', '59', '25', '60', '1', '4', '2', '3', '5', '6', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '19', '61', '62', '63', '64', '65', '66', '67', '26', '68', '69', '70', '71', '72', '18', '73', '27', '86', '74', '95', '21', '87', '89', '76', '2040', '2043', '1010', '1020'

In [99]:
#print(pdfRegCodes.loc[pdfRegCodes.index.str.len()<3])
#for r in pdfRegCodes.loc[pdfRegCodes.index.str.len()<3].iteritems():

SessPet=cnange_tor_proxy()
pdfPetrolPrices=None
for r in pdfRegCodes.loc[pdfRegCodes.index=='77'].index.tolist():
    pdfPetrolPrices=pd.concat([get_month_prices(current_region=int(r), current_month=m, current_year=y, session=SessPet) for y in range(lStartYear, 2019) for m in range(1, 13) ], axis=1)
    #lst=[(r[0], m, y) for y in range(lStartYear, 2019) for m in range(1, 13) ]
    #print(lst)
    #for y in range(lStartYear, 2019):
    #    for m in range(1, 13):
    #        print(r[0], m, y)

print(pdfPetrolPrices)

new IP is 185.220.101.28
start read for 1 month; 2008 year; 77 region ... error reading 1 - sleep and wait 30 sec...  new IP is 176.10.99.200
error reading 2 - sleep and wait 60 sec...  new IP is 104.223.123.98
error reading 3 - sleep and wait 90 sec...  new IP is 197.231.221.211
error reading 4 - sleep and wait 120 sec...  new IP is 185.227.68.78
error reading 5 - sleep and wait 150 sec...  new IP is 37.187.129.166
fatal error reading!!!
start read for 2 month; 2008 year; 77 region ... error reading 1 - sleep and wait 30 sec...  new IP is 37.187.129.166
error reading 2 - sleep and wait 60 sec...  new IP is 195.176.3.19
error reading 3 - sleep and wait 90 sec...  new IP is 46.38.235.14
error reading 4 - sleep and wait 120 sec...  new IP is 79.137.79.167
error reading 5 - sleep and wait 150 sec...  new IP is 144.217.80.80
fatal error reading!!!
start read for 3 month; 2008 year; 77 region ... error reading 1 - sleep and wait 30 sec...  new IP is 192.42.116.13
error reading 2 - sleep and

ConnectionError: SOCKSHTTPConnectionPool(host='icanhazip.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSConnection object at 0x7faed251a588>: Failed to establish a new connection: 0x01: General SOCKS server failure',))

In [ ]:
petrol=FuelsDataFrame(pdfPetrolPrices)
petrol.index.name='Date'
petrol=petrol.sort_index_date(inplace=False)

petrol.to_sql('PETROL_RU', eng_co, if_exists='upsert')
print(petrol.tail(5))